In [143]:
%%capture
# load the data into two raws and datasets ( capture - do not print to screen)

files1 = glob('/Users/shiran/workspace/muse-lsl/data/visual/P300/shiran/muse_monitor_dogs/data_2017-07-29-13.09.47.csv')
files2 = glob("/Users/shiran/workspace/muse-lsl/data/visual/P300/shiran/muse_monitor_dogs/data_2017-07-29-10.09.16.csv" )
files3 = glob("/Users/shiran/workspace/muse-lsl/data/visual/P300/shiran/muse_monitor_dogs/data_2017-07-29-13.21.06.csv" )


# files1 = glob('/Users/shiran/workspace/muse-lsl/data/visual/P300/shiran/dogs/data_2017-07-28-10.48*.csv')
# files2 = glob("/Users/shiran/workspace/muse-lsl/data/visual/P300/shiran/dogs/data_2017-07-29-09.26.*.csv" )

raw1 = get_raw(files2)
raw2 = get_raw(files3)

df1 = raw1.to_data_frame()
df2 = raw2.to_data_frame()


event_id = {'Non-Target': 1, 'Target': 2}
reject = {'eeg': 100e-6}
avg1, epochs1 = clean_epochs(event_id, raw1,{'start': 1,'end': 30}, None)
avg2, epochs2 = clean_epochs(event_id, raw2, {'start': 1,'end': 30}, reject)

In [144]:
from IPython.display import display
# display(df1.tail(10))
# display(df1[df1.Stim==1].describe())

display(df1.describe())
display(df2.describe())
display(df2.describe() - df1.describe())

signal,TP9,AF7,AF8,TP10,Stim
count,30336.000000,30336.000000,30336.000000,30336.000000,30336.000000
mean,846.611825,844.818058,839.981380,847.848964,0.006824
std,46.146971,22.572088,14.544283,41.975142,0.089610
min,610.440000,202.271000,749.451000,595.531000,0.000000
25%,802.637000,839.707000,834.469000,807.875000,0.000000
50%,844.542000,848.169000,844.139000,846.154000,0.000000
75%,890.879000,855.018000,849.377000,888.059000,0.000000
max,981.136000,886.044000,866.703000,1065.348000,2.000000


signal,TP9,AF7,AF8,TP10,Stim
count,30528.000000,30528.000000,30528.000000,30528.000000,30528.000000
mean,846.910118,850.177596,847.827110,844.119077,0.006748
std,65.079309,5.659600,4.398002,40.045141,0.088410
min,609.634000,813.516000,821.575000,634.212000,0.000000
25%,783.700000,846.557000,844.945000,806.667000,0.000000
50%,844.139000,850.586000,847.766000,842.527000,0.000000
75%,910.623000,853.810000,850.586000,882.115750,0.000000
max,1002.088000,893.297000,883.223000,953.333000,2.000000


signal,TP9,AF7,AF8,TP10,Stim
count,192.000000,192.000000,192.00000,192.000000,192.000000
mean,0.298293,5.359537,7.84573,-3.729887,-0.000076
std,18.932337,-16.912488,-10.14628,-1.930001,-0.001200
min,-0.806000,611.245000,72.12400,38.681000,0.000000
25%,-18.937000,6.850000,10.47600,-1.208000,0.000000
50%,-0.403000,2.417000,3.62700,-3.627000,0.000000
75%,19.744000,-1.208000,1.20900,-5.943250,0.000000
max,20.952000,7.253000,16.52000,-112.015000,0.000000


In [ ]:
from IPython.display import display
raw_data_uncliean=get_raw( files2)
#beofre cleaning
raw_data=get_raw(files2)
raw_data.filter(1,30)
display(raw_data.to_data_frame().describe())
display(raw_data_uncliean.to_data_frame().describe())

In [ ]:
display(raw1.plot_psd(tmax=np.inf))
raw2.plot_psd(tmax=np.inf)

In [ ]:
avg1.plot()
avg1.plot_image()

In [ ]:
# %load load_data.py
from glob import glob
import mne
from mne import create_info, concatenate_raws, Epochs, find_events
from mne.io import RawArray
from mne.channels import read_montage
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt


def describe_data(files):
    if len(files) == 0:
        print "no files found"
        return
    raw = get_raw(files)
    df = raw.to_data_frame()
    event_id = {'Non-Target': 1, 'Target': 2}
    reject = {'eeg': 100e-6}
    avg, epochs = clean_epochs(event_id, raw, {'start': 1, 'end': 30}, reject)

    df.describe()
    avg.plot()
    avg.plot_image()

    for kind in event_id.keys():
        subset = epochs[kind]
        av = subset.average()
        ixs_plot = mne.pick_types(av.info, meg=False, eeg=True)
        _ = av.plot_joint(picks=ixs_plot, title=kind)


def get_raw(files):
    raw = []
    for file_name in files:
        # read the file
        data = pd.read_csv(file_name, index_col=0)
        sfreq = 256

        # name of each channels
        ch_names = list(data.columns)[0:4] + ['Stim']

        # type of each channels
        ch_types = ['eeg'] * 4 + ['stim']
        montage = read_montage('standard_1005')

        # get data and exclude Aux channel
        data = data.values[:, [0, 1, 2, 3, 5]].T

        # convert to Volts (from uVolts (microvolt) to volt
        # from the docs:
        # /muse/eeg is uV , floats, range: 0.0 - 1682.815 uV
        # 1uv == (10^-6) * (1 Volt)
        data[:-1] *= 1e-6

        # create mne objects
        info = create_info(ch_names=ch_names, ch_types=ch_types, sfreq=sfreq, montage=montage)
        raw.append(RawArray(data=data, info=info))

    # concatenates all raw objects
    raw = concatenate_raws(raw)
    return raw


def clean_epochs(event_id, raw, filter, reject):
    # filter out events outside of start, end
    #  power line noise - 50 hz,
    if filter is not None:
        raw.filter(filter['start'], filter['end'], method='iir')
    events = find_events(raw)

    epochs = Epochs(raw, events=events, event_id=event_id, tmin=-0.1, tmax=0.8, baseline=None,
                    reject=reject, preload=True, verbose=False, picks=[0, 1, 2, 3], add_eeg_ref=False)

    avg = epochs.average()
    avg.plot()

    return avg, epochs


def p300(epochs):
    epochs.pick_types(eeg=True)
    X = epochs.get_data() * 1e6
    times = epochs.times
    y = epochs.events[:, -1]

    sns.set_context('talk')
    sns.set_style('white')
    fig, axes = plt.subplots(2, 2, figsize=[12, 6], sharex=True, sharey=True)
    axes = [axes[1, 0], axes[0, 0], axes[0, 1], axes[1, 1]]

    for ch in range(4):
        sns.tsplot(X[y == 1, ch], time=times, n_boot=1000, ax=axes[ch])
        sns.tsplot(X[y == 2, ch], time=times, color='r', n_boot=1000, ax=axes[ch])
        axes[ch].set_title(epochs.ch_names[ch])
        axes[ch].set_ylim(-6, 6)

    axes[0].set_xlabel('Time (s)')
    axes[0].set_ylabel('Amplitude (uV)')
    axes[-1].set_xlabel('Time (s)')
    axes[1].set_ylabel('Amplitude (uV)')
    plt.legend(['Non-target', 'Target'])
    sns.despine()
    plt.tight_layout()
    return plt


def p300_from_path(path, reject):
    files = glob(path)
    raw = get_raw(files)
    event_id = {'Non-Target': 1, 'Target': 2}
    avg, epochs = clean_epochs(event_id, raw, {'start': 1, 'en': 30}, reject)
    return p300(epochs)


In [ ]:
avg2.plot()
avg2.plot_image()

In [ ]:
plt1 = p300(epochs1)
plt1.show()

plt2 = p300(epochs2)
plt2.show()

In [ ]:
# this block enales to hide the code block
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="toggle on/off the raw code."></form>''')